In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [14]:
config_file = '../config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [61]:
m_gtf = '../'+expand(config['ref']['talon']['gtf'], species='human')[0]
m_ab_file = '../'+expand(config['lr']['talon']['ab'], species='human')[0]
m_filt_ab_file = '../'+expand(config['lr']['talon']['filt_ab'], species='human')[0]

In [62]:
df2 = pd.read_csv(m_filt_ab_file, sep='\t')
df2[['annot_transcript_id', 'gene_novelty']].groupby('gene_novelty').count()

,annot_transcript_id
gene_novelty,
Antisense,9795
Fusion,577
Intergenic,742
Known,180287


In [68]:
df2.loc[df2.gene_novelty=='Fusion'].head(15).tail(5)

,gene_ID,transcript_ID,annot_gene_id,annot_transcript_id,annot_gene_name,annot_transcript_name,n_exons,length,gene_novelty,transcript_novelty,...,h9_chondro_1_3,h9_panc_progen_1_2,hepg2_2_1,hl60_m1_12hr_1_2,hl60_m2_72hr_1_2,k562_3_2,lower_lobe_of_right_lung_1_1,ovary_2_1,pgp1_endo_1_1,right_cardiac_atrium_3_1
99127,62815,234794,TALONG000062815,TALONT000234794,TALONG000062815,TALONT000234794,5,1540,Fusion,ISM,...,1,5,5,1,1,16,4,7,4,6
99485,63684,238972,TALONG000063684,TALONT000238972,TALONG000063684,TALONT000238972,7,3704,Fusion,Fusion,...,0,0,0,0,0,1,3,8,0,0
99498,63684,239129,TALONG000063684,TALONT000239129,TALONG000063684,TALONT000239129,5,3678,Fusion,ISM,...,14,0,0,6,2,3,0,0,1,0
101261,66573,259108,TALONG000066573,TALONT000259108,TALONG000066573,TALONT000259108,13,3250,Fusion,ISM,...,0,0,0,0,0,0,1,0,0,0
101549,67522,262589,TALONG000067522,TALONT000262589,TALONG000067522,TALONT000262589,2,3645,Fusion,ISM,...,0,0,0,0,0,0,0,0,0,0


In [66]:
df2.loc[(df2.annot_transcript_id=='TALONT000214728')|(df2.annot_gene_name=='BRD9')]

,gene_ID,transcript_ID,annot_gene_id,annot_transcript_id,annot_gene_name,annot_transcript_name,n_exons,length,gene_novelty,transcript_novelty,...,h9_chondro_1_3,h9_panc_progen_1_2,hepg2_2_1,hl60_m1_12hr_1_2,hl60_m2_72hr_1_2,k562_3_2,lower_lobe_of_right_lung_1_1,ovary_2_1,pgp1_endo_1_1,right_cardiac_atrium_3_1
24936,14914,54193,ENSG00000028310.17,ENST00000497410.5,BRD9,BRD9-214,2,1916,Known,Known,...,0,0,0,1,0,0,1,0,1,0
24937,14914,54194,ENSG00000028310.17,ENST00000483234.5,BRD9,BRD9-205,6,2211,Known,Known,...,0,1,0,1,1,1,0,0,0,0
24938,14914,54197,ENSG00000028310.17,ENST00000495265.5,BRD9,BRD9-212,16,2843,Known,Known,...,0,1,0,0,0,0,0,2,0,0
24939,14914,54199,ENSG00000028310.17,ENST00000483173.5,BRD9,BRD9-204,16,2120,Known,Known,...,0,0,0,0,0,0,0,0,0,0
24940,14914,54200,ENSG00000028310.17,ENST00000467963.5,BRD9,BRD9-202,16,2156,Known,Known,...,11,0,7,2,17,23,11,13,8,13
24941,14914,54201,ENSG00000028310.17,ENST00000519112.5,BRD9,BRD9-217,7,861,Known,Known,...,0,0,0,0,0,0,0,0,0,0
97352,14914,214706,ENSG00000028310.17,TALONT000214706,BRD9,TALONT000214706,16,2724,Known,NIC,...,4,0,1,0,2,10,6,8,2,2
97354,59968,214728,TALONG000059968,TALONT000214728,TALONG000059968,TALONT000214728,16,2661,Fusion,ISM,...,3,0,1,0,2,8,2,3,2,3
97362,14914,214877,ENSG00000028310.17,TALONT000214877,BRD9,TALONT000214877,8,2918,Known,ISM,...,0,2,0,3,2,3,2,1,0,2
97367,14914,214926,ENSG00000028310.17,TALONT000214926,BRD9,TALONT000214926,12,2055,Known,ISM,...,0,2,0,3,0,0,0,0,1,0


In [48]:
gtf_df = pr.read_gtf(m_gtf, duplicate_attr=True).df
gtf_df['tag'] = gtf_df.tag.astype(str)


In [49]:
gtf_df.loc[gtf_df.Chromosome=='chr1']

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
772,chr1,HAVANA,gene,11868,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN
773,chr1,HAVANA,transcript,11868,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,NaN,NaN,NaN,NaN,NaN
774,chr1,HAVANA,exon,11868,12227,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,1,ENSE00002234944.1,NaN,NaN,NaN
775,chr1,HAVANA,exon,12612,12721,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,2,ENSE00003582793.1,NaN,NaN,NaN
776,chr1,HAVANA,exon,13220,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,3,ENSE00002312635.1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251291,chr1,HAVANA,exon,248858917,248859033,.,-,.,ENSG00000171163.15,protein_coding,...,retained_intron,ZNF692-228,4,nan,OTTHUMT00000382606.1,1,ENSE00002188413.1,NaN,NaN,NaN
251292,chr1,HAVANA,exon,248857974,248858385,.,-,.,ENSG00000171163.15,protein_coding,...,retained_intron,ZNF692-228,4,nan,OTTHUMT00000382606.1,2,ENSE00002170424.1,NaN,NaN,NaN
251293,chr1,ENSEMBL,gene,248912689,248912795,.,-,.,ENSG00000200495.1,snRNA,...,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN
251294,chr1,ENSEMBL,transcript,248912689,248912795,.,-,.,ENSG00000200495.1,snRNA,...,snRNA,RNU6-1205P-201,NA,basic,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
# are all "overlapping_locus" tags just genes?
gtf_df.loc[(gtf_df.tag.str.contains('overlapping_locus'))&(gtf_df.Feature!='gene')]
# yes

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid


In [51]:
# what entries have the "overlapping_locus" tag?
temp = gtf_df.loc[gtf_df.tag.str.contains('overlapping_locus')].copy(deep=True)
temp[['tag', 'Feature']].groupby('Feature').count()

,tag
Feature,
gene,10451


In [52]:
# there are no entries with the "readthrough_gene" tag
gtf_df.loc[gtf_df.tag.str.contains('readthrough_gene')]

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid


In [53]:
# what entries have the "readthrough_transcript" tag?
temp = gtf_df.loc[gtf_df.tag.str.contains('readthrough_transcript')].copy(deep=True)
temp[['tag', 'Feature']].groupby('Feature').count()

,tag
Feature,
CDS,6706
UTR,5761
exon,15759
start_codon,804
stop_codon,974
transcript,1879


In [54]:
# get a list of genes w/ the overlapping_locus tags
o_genes = gtf_df.loc[gtf_df.tag.str.contains('overlapping_locus'), 'gene_id'].unique().tolist()

# how many transcripts from genes w/ overlapping locus tag have the 'read_trhough_transcript' tag?
temp = gtf_df.loc[gtf_df.gene_id.isin(o_genes)].copy(deep=True)
temp = temp.loc[temp.tag.str.contains('readthrough_transcript')].copy(deep=True)
temp[['tag', 'Feature']].groupby('Feature').count()

# so there are a few transcrpts with the readthrough_transcript tag that 
# don't belong to the overlapping_locus genes

,tag
Feature,
CDS,6490
UTR,5640
exon,15397
start_codon,775
stop_codon,945
transcript,1833


In [55]:
# more throughough check for concordance of overlapping locus and 
temp = gtf_df.loc[gtf_df.Feature=='transcript'][['gene_id', 'gene_name', 'tag', 'transcript_id']].copy(deep=True)
temp['readthrough_transcript'] = temp.tag.str.contains('readthrough_transcript')
temp.drop('tag', inplace=True, axis=1)
temp.drop_duplicates(inplace=True)

temp2 = gtf_df.loc[gtf_df.Feature=='gene'][['gene_id', 'tag']].copy(deep=True)
temp2['overlapping_locus'] = temp2.tag.str.contains('overlapping_locus')
temp2.drop('tag', axis=1, inplace=True)
temp2.drop_duplicates(inplace=True)

temp = temp.merge(temp2, how='outer', on='gene_id')

In [56]:
temp.groupby(['readthrough_transcript', 'overlapping_locus']).count().reset_index()

,readthrough_transcript,overlapping_locus,gene_id,gene_name,transcript_id
0,False,False,155960,155780,155960
1,False,True,49102,49102,49102
2,True,False,46,46,46
3,True,True,1833,1833,1833


In [57]:
temp.loc[(temp.readthrough_transcript==True)&(temp.overlapping_locus==False)].head()

,gene_id,gene_name,transcript_id,readthrough_transcript,overlapping_locus
1447,ENSG00000285721.1,AL031281.2,ENST00000648594.1,True,False
5516,ENSG00000280778.1,AC243547.3,ENST00000630636.1,True,False
5517,ENSG00000280778.1,AC243547.3,ENST00000625258.1,True,False
12653,ENSG00000285839.1,AL445685.3,ENST00000648686.1,True,False
16180,ENSG00000158887.15,MPZ,ENST00000533357.4,True,False


In [58]:
temp.loc[(temp.readthrough_transcript==False)&(temp.overlapping_locus==True)].head()

,gene_id,gene_name,transcript_id,readthrough_transcript,overlapping_locus
204,ENSG00000240409.1,MTATP8P1,ENST00000467115.1,False,True
205,ENSG00000248527.1,MTATP6P1,ENST00000514057.1,False,True
230,ENSG00000285268.1,AL669831.7,ENST00000644482.1,False,True
248,ENSG00000272438.1,AL645608.6,ENST00000607769.1,False,True
249,ENSG00000230699.2,AL645608.2,ENST00000448179.1,False,True


In [60]:
temp.loc[temp.gene_name=='MPZ']

,gene_id,gene_name,transcript_id,readthrough_transcript,overlapping_locus
16179,ENSG00000158887.15,MPZ,ENST00000463290.5,False,False
16180,ENSG00000158887.15,MPZ,ENST00000533357.4,True,False
16181,ENSG00000158887.15,MPZ,ENST00000491222.5,False,False
16182,ENSG00000158887.15,MPZ,ENST00000526189.2,False,False
16183,ENSG00000158887.15,MPZ,ENST00000476410.1,False,False
16184,ENSG00000158887.15,MPZ,ENST00000488271.1,False,False


In [59]:
temp.loc[temp.gene_name=='CENPS-CORT']

,gene_id,gene_name,transcript_id,readthrough_transcript,overlapping_locus
844,ENSG00000251503.8,CENPS-CORT,ENST00000400900.6,True,True
845,ENSG00000251503.8,CENPS-CORT,ENST00000470413.6,True,True
846,ENSG00000251503.8,CENPS-CORT,ENST00000602787.6,True,True
847,ENSG00000251503.8,CENPS-CORT,ENST00000602296.6,True,True
848,ENSG00000251503.8,CENPS-CORT,ENST00000465026.1,True,True


## looking at abundance mats

In [4]:
df = pd.read_csv(m_ab_file, sep='\t')

In [6]:
df.gene_novelty.unique()

array(['Known', 'Antisense', 'Intergenic', 'Fusion'], dtype=object)

In [7]:
df[['annot_gene_id', 'gene_novelty']].groupby('gene_novelty').count().reset_index()

,gene_novelty,annot_gene_id
0,Antisense,222873
1,Fusion,43465
2,Intergenic,64034
3,Known,2552024


In [8]:
df.loc[df.gene_novelty=='Fusion']

,gene_ID,transcript_ID,annot_gene_id,annot_transcript_id,annot_gene_name,annot_transcript_name,n_exons,length,gene_novelty,transcript_novelty,...,adrenal_gland_1_2,cortex_5x_f_1_2,cortex_14d_f_2,cortex_2mo_m_2,gastroc_14d_f_2,gastroc_25d_m_2,gastroc_4d_f_2,heart_18-20mo_m_1,hippocampus_wt_m_2_1,hippocampus_18-20mo_f_2
64257,55774,142467,TALONG000055774,TALONT000142467,TALONG000055774,TALONT000142467,13,1934,Fusion,Fusion,...,0,0,0,0,0,0,0,0,0,0
64383,55808,142593,TALONG000055808,TALONT000142593,TALONG000055808,TALONT000142593,9,3331,Fusion,Fusion,...,0,1,1,0,0,0,0,0,2,1
64754,55869,142964,TALONG000055869,TALONT000142964,TALONG000055869,TALONT000142964,8,2352,Fusion,Fusion,...,0,2,0,0,0,0,0,0,1,0
64757,55869,142967,TALONG000055869,TALONT000142967,TALONG000055869,TALONT000142967,9,2518,Fusion,NIC,...,0,3,0,0,0,0,0,0,0,0
64759,55869,142969,TALONG000055869,TALONT000142969,TALONG000055869,TALONT000142969,4,1035,Fusion,NIC,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2882153,127991,2960363,TALONG000127991,TALONT002960363,TALONG000127991,TALONT002960363,1,1207,Fusion,Genomic,...,1,0,0,0,1,0,0,0,0,0
2882155,127991,2960365,TALONG000127991,TALONT002960365,TALONG000127991,TALONT002960365,6,1507,Fusion,NNC,...,0,0,0,0,0,0,0,0,1,0
2882291,180891,2960501,TALONG000180891,TALONT002960501,TALONG000180891,TALONT002960501,18,3855,Fusion,NIC,...,0,0,0,1,0,0,0,0,0,0
2882323,180891,2960533,TALONG000180891,TALONT002960533,TALONG000180891,TALONT002960533,1,1894,Fusion,Genomic,...,1,0,0,0,0,0,0,0,0,0
